# 4. Networks Using Blocks (VGG)

In [3]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn

## Visual Geometry Group

The **basic building block** of CNNs is a sequence of the following: 

1. convolutional layers with padding (to maintain the resolution)
2. non-linear activation functions
3. pooling layers (for spatial downsampling)

In [4]:
def vgg_block(num_convs, in_channels, out_channels):
    layers = []
    for _ in range(num_convs):
        layers.append(nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1))
        layers.append(nn.ReLU())
        in_channels = out_channels
    layers.append(nn.MaxPool2d(kernel_size=2,stride=2))
    return nn.Sequential(*layers)

## VGG Networks

The following compares a **VGG Network** with the AlexNet:

![](http://d2l.ai/_images/vgg.svg)

The **original VGG** consists of **5 VGG blocks**:

1. block 1 and 2: each contains 1 convolutional layer
2. block 3, 4, and 5: each contains 2 convolution layers

Since this VGG has 8 convolutional layers and 3 fully connected layers, it is often called VGG-11.

We use a **hyperparameter** conv_arch to define the number of convolutional layers and output channels in each VGG block:

In [5]:
#(number of convolutional layers, number of output channels)
conv_arch = ((1, 64), (1, 128), (2, 256), (2, 512), (2, 512))

In [6]:
def vgg(conv_arch):
    
    in_channels = 1

    conv_blks = []
    for (num_convs, out_channels) in conv_arch:
        conv_blks.append(vgg_block(num_convs, in_channels, out_channels))
        in_channels = out_channels

    return nn.Sequential(*conv_blks, 
                         nn.Flatten(),
                         nn.Linear(in_features=out_channels*7*7, out_features=4096), nn.ReLU(), 
                         nn.Dropout(0.5),
                         nn.Linear(in_features=4096, out_features=4096), nn.ReLU(), 
                         nn.Dropout(0.5),
                         nn.Linear(in_features=4096, out_features=10))

In [7]:
net = vgg(conv_arch)

In [8]:
X = torch.randn(size=(1, 1, 224, 224))

for blk in net:
    X = blk(X)
    print(blk.__class__.__name__,'output shape:\t',X.shape)

Sequential output shape:	 torch.Size([1, 64, 112, 112])
Sequential output shape:	 torch.Size([1, 128, 56, 56])
Sequential output shape:	 torch.Size([1, 256, 28, 28])
Sequential output shape:	 torch.Size([1, 512, 14, 14])
Sequential output shape:	 torch.Size([1, 512, 7, 7])
Flatten output shape:	 torch.Size([1, 25088])
Linear output shape:	 torch.Size([1, 4096])
ReLU output shape:	 torch.Size([1, 4096])
Dropout output shape:	 torch.Size([1, 4096])
Linear output shape:	 torch.Size([1, 4096])
ReLU output shape:	 torch.Size([1, 4096])
Dropout output shape:	 torch.Size([1, 4096])
Linear output shape:	 torch.Size([1, 10])
